In [ ]:
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow as tf
import os
import shutil

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.data.experimental import AUTOTUNE
from tensorflow import keras

In [ ]:
sleeves = ['long', 'invisible', 'elbow', 'short', 'sleeveless']
for sleeve in sleeves:
    #os.mkdir(f"sleeve/{sleeve}")
    print(sleeve,": ",len(os.listdir(f"sleeve/{sleeve}")))

In [81]:
def dataAug(train_ds, num=5):
    aug1 = layers.experimental.preprocessing.RandomFlip("horizontal")
    ds_1 = train_ds.map(lambda x, y: (aug1(x), y), num_parallel_calls=AUTOTUNE)
    aug2 = layers.experimental.preprocessing.RandomRotation(0.2)
    ds_2 = train_ds.map(lambda x, y: (aug2(x), y), num_parallel_calls=AUTOTUNE)
    aug3 = layers.RandomContrast(0.4)
    ds_3 = train_ds.map(lambda x, y: (aug3(x), y), num_parallel_calls=AUTOTUNE)
    aug4 = layers.RandomBrightness(0.4)
    ds_4 = train_ds.map(lambda x, y: (aug4(x), y), num_parallel_calls=AUTOTUNE)
    aug5 = layers.RandomTranslation(0.3, 0.3)
    ds_5 = train_ds.map(lambda x, y: (aug5(x), y), num_parallel_calls=AUTOTUNE)
    for i in range(num):
        train_ds = train_ds.concatenate(locals()[f'ds_{i+1}'])
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    return train_ds

In [82]:
def Split(directory, validation = 0.3, inception = False):  
    if inception == False:
        IMG_DIM = 224     
    else:
        IMG_DIM = 299       # for Inception ResNet v2
    
    BATCH_SIZE = 64

    train_ds = image_dataset_from_directory(directory,
                                                image_size=(IMG_DIM,IMG_DIM),
                                                label_mode="categorical",
                                                batch_size=BATCH_SIZE,
                                                validation_split = validation,
                                                subset="training",
                                                seed=117,
                                                shuffle=True)

    val_ds = image_dataset_from_directory(directory,
                                                image_size=(IMG_DIM,IMG_DIM),
                                                label_mode="categorical",
                                                batch_size=BATCH_SIZE,
                                                validation_split = validation,
                                                subset="validation",
                                                seed=117,
                                                shuffle=True)
    
    

    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

    return train_ds, val_ds

In [83]:
def RUN(train_ds, val_ds, category, inception = False, trainable = False, epoch = 60, earlystop = 10):
    ## Build the model
    n_classes = train_ds.element_spec[1].shape[-1]

    if inception == False:
        IMG_DIM = 224      # for ResNet v2
        basemodel_path = "https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5"
    else:
        IMG_DIM = 299       # for Inception ResNet v2
        basemodel_path = "https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/5"
   
    model = Sequential([
    
        hub.KerasLayer(basemodel_path,
        trainable = trainable,
        arguments = dict(batch_norm_momentum = 0.997)),
        layers.Dropout(0.4),
        layers.Flatten(),
        layers.Dense(n_classes, activation='softmax')
        ])
    model.build([None, IMG_DIM, IMG_DIM, 3])
    model.summary()

    ## Complie the model
    model.compile(optimizer='adam',
              loss="categorical_crossentropy",     # SparseCategoricalCrossentropy used for interger Yi; CategoricalCrossentropy used for one-hot Yi
              metrics=['accuracy'])
    
    ## Fit the model
    epochs = epoch

    checkpoint = ModelCheckpoint(f"weights_{category}.hdf5", monitor='val_accuracy', mode="max", verbose = 1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=earlystop)
    callback_list = [checkpoint, early_stopping]

    history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callback_list
    )

    return history

In [84]:
def Continue(train_ds, val_ds, category, model_path, epoch=60, earlystop=10):
    model = keras.models.load_model(model_path, custom_objects={'KerasLayer':hub.KerasLayer})
    model.summary()

    ## Complie the model
    model.compile(optimizer='adam',
              loss="categorical_crossentropy",     # SparseCategoricalCrossentropy used for interger Yi; CategoricalCrossentropy used for one-hot Yi
              metrics=['accuracy'])
    
    ## Fit the model
    epochs = epoch

    checkpoint = ModelCheckpoint(f"weights_{category}.hdf5", monitor='val_accuracy', mode="max", verbose = 1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=earlystop)
    callback_list = [checkpoint, early_stopping]

    history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callback_list
    )

    return history

In [85]:
def Vis(category, *history):
    acc=[]
    val_acc=[]
    loss=[]
    val_loss=[]
    
    for hist in history:
        acc.append(hist.history['accuracy'])
        val_acc.append(hist.history['val_accuracy'])
        loss.append(hist.history['loss'])
        val_loss.append(hist.history['val_loss'])

    epochs_range = range(len(acc))

    plt.figure(figsize=(16, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title(f'{category}: Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title(f'{category}: Training and Validation Loss')
    plt.savefig(f'{category}.png')
    plt.show()


In [75]:
print_train, print_val = Split('sleeve', 0.4)

Found 26464 files belonging to 5 classes.
Using 15879 files for training.
Found 26464 files belonging to 5 classes.
Using 10585 files for validation.


In [76]:
print(len(print_train))
print(len(print_val))

249
166


In [77]:
print_train = dataAug(print_train, 5)
#print_val = dataAug(print_val, 3)
print(len(print_train))
#print(len(print_val))

1494


In [ ]:
model.save("print_model_aug")

In [57]:
model = keras.models.load_model("weights_Print_Design.hdf5", custom_objects={'KerasLayer':hub.KerasLayer})
model.summary()

TypeError: weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.

In [ ]:
shutil.make_archive("print_model_aug", "zip", "print_model_aug")

In [ ]:
print_hist = Continue(print_train, print_val, 'Print', "weights_Print.hdf5", 50, 15)

In [ ]:
print_hist = RUN(print_train, print_val, 'Print_1', inception = False, trainable = True, epoch = 50, earlystop = 15)

In [ ]:
RUN(train_ds, val_ds, category, inception = False, trainable = False, epoch = 60, earlystop = 10):